### Tutorial do wczytywania modeli

Poniżej są biblioteki, które zostały użyte do trenowania modelu. Raczej pasuje je mieć.

In [104]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import lightning.pytorch as pl
import pandas as pd
import shutil

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


Poniżej jest kilka parametrów, tylkoo niektóre są użyte do zdefiniowania modelu, ale skopiowałem wszystkie z oryginalnego pliku, bo jestem leniwy.

In [105]:

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 128

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 32

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

Żeby wczytać model konieczne jest odtworzenie jego struktury, ZANIM się go wczyta. Stąd też potrzeba powyższych parametrów. Nwm czy metoda forward jest potrzebna, obstawiam że tak.

In [106]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, stride=2, padding=1, bias=False), 
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 16 x 16 
            nn.Conv2d(ndf * 4, ndf * 8, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 8 x 8
            nn.Conv2d(ndf * 8, ndf * 16, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x 4 x 4
            nn.Conv2d(ndf * 16, 1, 4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
            # state size. 1
        )

    def forward(self, input):
        return self.main(input)

Poniżej wczytanie modelu z pliku. Wywołanie .eval() jest obowiązkowe (jakieś tam pierdolenie z batch normalization i dropoutami, nwm nie znam się).

In [107]:
netD = torch.load('discriminator_true.pth',map_location=torch.device('cpu'))
netD.eval()

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (12): BatchNorm2d(512, eps=1e-05, mom

### Tworzenie folderów real i fake
Do data_path dodaj ścieżkę do folderu z plikami .flac <br>
Skrypt spakuje te spektrogramy do folderów: reals oraz fakes na podstawie pliku CSV ASVspoof.

In [108]:
# Get real/fake key from CSV
data_key = pd.read_csv('spoof_data.csv')
data_real_ID_list = data_key[data_key['spoof_bonafide']=='bonafide']
data_fake_ID_list = data_key[data_key['spoof_bonafide']=='spoof']

# split data to seperate folders containing fakes and reals and combined data
data_path = '../spectrograms/real_noise/'
data_reals_path = os.path.join(data_path,'reals')
data_fakes_path = os.path.join(data_path,'fakes')
if not os.path.exists(data_reals_path):
    os.makedirs(data_reals_path)
if not os.path.exists(data_fakes_path):
    os.makedirs(data_fakes_path)

if len(os.listdir(data_reals_path)) == 0:
    for filename in os.listdir(data_path):
        file_path = os.path.join(data_path, filename)

        # checking if it is a file
        if os.path.isfile(file_path):

            filename = filename[:-4]
            # check if file is fake or real
            if(filename in data_real_ID_list['recording_ID'].values):
                shutil.move(file_path, data_reals_path)
            elif(filename in data_fake_ID_list['recording_ID'].values):
                shutil.move(file_path, data_fakes_path)
            else:
                print(f'No file with name {filename} found in key database!')

### Tworzenie datasetów mixed, real oraz fake
tworzymy jeden dataset ze wszystkimi obrazami i dodatkowe dwa subsety z real'ami i fake'ami. 

In [109]:
# Create the mixed dataset
dataset_mixed = dset.ImageFolder(root=data_path,
                                transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the mixed dataloader
dataloader_mixed = torch.utils.data.DataLoader(dataset_mixed, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Create fake/real subsets
list_of_real_idx = []
list_of_fake_idx = []

for num,(file_path, idx) in enumerate(dataset_mixed.imgs):
    dir_path = file_path[:-17]
    if(dir_path == data_reals_path):
        list_of_real_idx.append(num)
    elif(dir_path == data_fakes_path):
        list_of_fake_idx.append(num)
    else:
        print('Invalid data!')

subset_real = torch.utils.data.Subset(dataset=dataset_mixed, indices=list_of_real_idx)
# Create the reals dataloader
dataloader_reals = torch.utils.data.DataLoader(dataset=subset_real, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

subset_fake = torch.utils.data.Subset(dataset=dataset_mixed, indices=list_of_fake_idx)
# Create the fakes dataloader
dataloader_fakes = torch.utils.data.DataLoader(dataset=subset_fake, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)


# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

### Ewaluacja modelu dla danych mixed

In [110]:
num_correct = 0
num_samples = 0

with torch.no_grad():
    for x, y in dataloader_mixed:
        x = x.to(device=device)
        y = y.to(device=device)
        
        scores = netD(x)
        _, predictions = scores.max(1)
        num_samples += predictions.size(0)
        num_correct += (torch.squeeze(predictions) == y).sum()
    
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

Got 3275 / 3406 with accuracy 96.15


### Ewaluacja modelu dla danych real

In [116]:
num_correct = 0
num_samples = 0

with torch.no_grad():
    for x, y in dataloader_reals:
        x = x.to(device=device)
        y = y.to(device=device)
        
        scores = netD(x)
        _, predictions = scores.max(1)
        num_samples += predictions.size(0)
        num_correct += (torch.squeeze(predictions) == y).sum()
    
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

Got 0 / 131 with accuracy 0.00


### Ewaluacja modelu dla danych fake

In [112]:
num_correct = 0
num_samples = 0

with torch.no_grad():
    for x, y in dataloader_fakes:
        x = x.to(device=device)
        y = y.to(device=device)
        
        scores = netD(x)
        _, predictions = scores.max(1)
        num_samples += predictions.size(0)
        num_correct += (torch.squeeze(predictions) == y).sum()
    
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

Got 3275 / 3275 with accuracy 100.00
